## Install Requirements and Download Dataset

In [ ]:
!pip install -r requirements.txt
!wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar -xzf lastfm-dataset-1K.tar.gz

## Imports

In [ ]:
import os 
import pickle
import random
import statistics
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
from itertools import islice
from mpl_toolkits.mplot3d import Axes3D
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from tensorflow.python.ops import tensor_array_ops, control_flow_ops

## Read In Last.fm Data

In [ ]:
def read_data(nRows=None):
    start_time = time.time()
    directory = "lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"
    df = pd.read_csv(directory, engine='python',
                          nrows=nRows, header=None, sep='\\t',
                          names=['userId', 'date', 'artist_id', 'artist_name', 'track_id', 'track_name'])
    df.dataframeName = 'userid-timestamp-artid-artname-traid-traname.tsv'
    df.date = pd.to_datetime(df.date)
    nRow, nCol = df.shape
    print(f'There are {nRow} rows and {nCol} columns')

    return df
    

## Data Exploration

In [ ]:
def summary_stats(df):
    nan_values = df.track_id.isna().sum()
    unique_tracks = len(df.track_name.unique())
    print("There are {} NaN track_ids and {} unique tracks".format(nan_values, unique_tracks))

In [ ]:
def find_repeated_tracks(session_frame):
    repeats = {}

    for row in session_frame.iloc[1]:
        print(row)
        i = 0

        while i < (len(row) - 1):
            if row[i] == row[i + 1]:
                if 2 in repeats:
                    repeats[2] += 1
                else:
                    repeats[2] = 1
                j = i + 1
                count = 3
                while j < len(row) - 1:
                    if row[j] == row[j + 1]:
                        if count in repeats:
                            repeats[count] += 1
                        else:
                            repeats[count] = 1
                        j += 1
                        count += 1
                    else:
                        i = j
                        break
            i += 1
    return repeats

In [ ]:
def print_top_tracks(all_tracks, nTracks=10):
    sorted_tracks = [(k, (all_tracks[k]['artist'], all_tracks[k]['track_name'], all_tracks[k]['plays'])) for k in
                     sorted(all_tracks, key=lambda x: all_tracks[x]['plays'], reverse=True)]

    n_items = list(islice(sorted_tracks, 10))

    print("\nTOP TRACKS")
    print("{:<5s} {:<40s} {:<40s} {:<6s}".format("Rank","Artist","Track","Plays"))

    for idx, tk in enumerate(n_items):
        print("{:<5d} {:<40s} {:<40s} {:<6d} ".format(idx + 1, tk[1][0], tk[1][1], tk[1][2]))

## Assign Unique track_ids

In [ ]:
def assign_unique_ids(df1):
    print("Assigning unique track ids...")
    unique_track_id = {}
    start_time = time.time()

    for idx, value in enumerate(df1.track_name.unique()):
        unique_track_id[value] = idx

    print(len(unique_track_id))
    print("Completed in {:0.2f} seconds".format(time.time() - start_time))
    
    return unique_track_id

## Get Sessions from Tracks

In [ ]:
def sessions_from_tracks(daniel_track_id, tracks, df, session_size):
    count=0
    track_list = []
    track_times = []

    sessions = pd.DataFrame(columns=['user_session_ID', 'session', 'start_time', 'length'])
    
    session_size = np.timedelta64(session_size, 'm') 
    prev_time = np.timedelta64(0, 'us')

    for idx, row in df.iterrows():
        t_id = daniel_track_id[row.track_name]
  #  Calculate track plays       
        if t_id not in tracks:
            tracks[t_id] = {'track_name': row.track_name,  'artist': row.artist_name, 'plays': 1}
        else:
            tracks[t_id]['plays'] += 1
        track_list.append(daniel_track_id[row.track_name])
        track_times.append(row.date)
        
        
        if((row.date - prev_time)> session_size):
            userID = (row.userId + '_session_' + str(count+1))
            sessions = sessions.append({'user_session_ID': userID, 'session': track_list, 'start_time': track_times[0], 'length': (row.date - track_times[0])}, ignore_index=True)
            count += 1
            track_list = []
            track_times = []
        
        prev_time = row.date
    
    return sessions

## Get Sessions per User

In [ ]:
def sessions_from_frame(df1, daniel_track_id, session_size):
    start_time = st = time.time()
    all_tracks = {}
    session_data = pd.DataFrame()
    session_frame = pd.DataFrame(columns=['user_session_ID', 'session', 'start_time', 'length'])

    for idx, group in df1.groupby('userId')[['date', 'track_id', 'track_name', 'artist_name']]:
        group = group.sort_values(by='date', ascending=True)
        group.date = pd.to_timedelta(group.date.astype('int64'), unit='ns')
        session_data = sessions_from_tracks(daniel_track_id, all_tracks, group, session_size)
        session_frame = session_frame.append(session_data, ignore_index=True)

        start_time=time.time()
      
    return session_frame, all_tracks

## Get Sessions

In [ ]:
def get_sessions(nRows):

    df = read_data(nRows)
    summary_stats(df)
    unique_ids = assign_unique_ids(df)
    session_frame, all_tracks = sessions_from_frame(df, unique_ids, session_size=20)
#     print_top_tracks(all_tracks)
    df_new = session_frame.session
    
    return df_new, all_tracks

## Model

In [ ]:
def _cumsum(x, length):
    lower_triangular_ones = tf.constant(
        np.tril(np.ones((length, length))),
        dtype=tf.float32)
    return tf.reshape(
            tf.matmul(lower_triangular_ones,
                      tf.reshape(x, [length, 1])),
            [length])


def _backwards_cumsum(x, length):
    upper_triangular_ones = tf.constant(
        np.triu(np.ones((length, length))),
        dtype=tf.float32)
    return tf.reshape(
            tf.matmul(upper_triangular_ones,
                      tf.reshape(x, [length, 1])),
            [length])


class RNN(object):

    def __init__(self, num_emb, emb_dim, hidden_dim,
                 sequence_length, start_token,
                 learning_rate=0.01, reward_gamma=0.9):
        self.num_emb = num_emb
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        self.start_token = tf.constant(start_token, dtype=tf.int32)
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.reward_gamma = reward_gamma
        self.g_params = []
        self.d_params = []

        self.expected_reward = tf.Variable(tf.zeros([self.sequence_length]))

        with tf.compat.v1.variable_scope('generator'):
            self.g_embeddings = tf.Variable(self.init_matrix([self.num_emb, self.emb_dim]))
            self.g_params.append(self.g_embeddings)
            self.g_recurrent_unit = self.create_recurrent_unit(self.g_params)  # maps h_tm1 to h_t for generator
            self.g_output_unit = self.create_output_unit(self.g_params, self.g_embeddings)  # maps h_t to o_t (output token logits)

        with tf.compat.v1.variable_scope('discriminator'):
            self.d_embeddings = tf.Variable(self.init_matrix([self.num_emb, self.emb_dim]))
            self.d_params.append(self.d_embeddings)
            self.d_recurrent_unit = self.create_recurrent_unit(self.d_params)  # maps h_tm1 to h_t for discriminator
            self.d_classifier_unit = self.create_classifier_unit(self.d_params)  # maps h_t to class prediction logits
            self.d_h0 = tf.Variable(self.init_vector([self.hidden_dim]))
            self.d_params.append(self.d_h0)

        self.h0 = tf.compat.v1.placeholder(tf.float32, shape=[self.hidden_dim])  # initial random vector for generator
        self.x = tf.compat.v1.placeholder(tf.int32, shape=[self.sequence_length])  # sequence of indices of true data, not including start token
        self.samples = tf.compat.v1.placeholder(tf.float32, shape=[self.sequence_length])  # random samples from [0, 1]

        # generator on initial randomness
        gen_o = tensor_array_ops.TensorArray(dtype=tf.float32, size=self.sequence_length,
                                             dynamic_size=False, infer_shape=True)
        gen_x = tensor_array_ops.TensorArray(dtype=tf.int32, size=self.sequence_length,
                                             dynamic_size=False, infer_shape=True)
        samples = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        samples = samples.unstack(self.samples)
        def _g_recurrence(i, x_t, h_tm1, gen_o, gen_x):
            h_t = self.g_recurrent_unit(x_t, h_tm1)
            o_t = self.g_output_unit(h_t)
            sample = samples.read(i)
            o_cumsum = _cumsum(o_t, self.num_emb)  # prepare for sampling ***
            next_token = tf.compat.v1.to_int32(tf.reduce_min(tf.where(sample < o_cumsum)))  # sample
            x_tp1 = tf.gather(self.g_embeddings, next_token)
            gen_o = gen_o.write(i, tf.gather(o_t, next_token))  # we only need the sampled token's probability
            gen_x = gen_x.write(i, next_token)  # indices, not embeddings
            return i + 1, x_tp1, h_t, gen_o, gen_x

        _, _, _, self.gen_o, self.gen_x = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3, _4: i < self.sequence_length,
            body=_g_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.gather(self.g_embeddings, self.start_token),
                       self.h0, gen_o, gen_x))

        # discriminator on generated and real data
        d_gen_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)
        d_real_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)

        self.gen_x = self.gen_x.stack()
        emb_gen_x = tf.gather(self.d_embeddings, self.gen_x)
        ta_emb_gen_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_gen_x = ta_emb_gen_x.unstack(emb_gen_x)

        emb_real_x = tf.gather(self.d_embeddings, self.x)
        ta_emb_real_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_real_x = ta_emb_real_x.unstack(emb_real_x)

        def _d_recurrence(i, inputs, h_tm1, pred):
            x_t = inputs.read(i)
            h_t = self.d_recurrent_unit(x_t, h_tm1)
            y_t = self.d_classifier_unit(h_t)
            pred = pred.write(i, y_t)
            return i + 1, inputs, h_t, pred

        _, _, _, self.d_gen_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_d_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       ta_emb_gen_x,
                       self.d_h0,
                       d_gen_predictions))
        self.d_gen_predictions = tf.reshape(
                self.d_gen_predictions.stack(),
                [self.sequence_length])

        _, _, _, self.d_real_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_d_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       ta_emb_real_x,
                       self.d_h0,
                       d_real_predictions))
        self.d_real_predictions = tf.reshape(
                self.d_real_predictions.stack(),
                [self.sequence_length])

        # supervised pretraining for generator
        g_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)

        emb_x = tf.gather(self.g_embeddings, self.x)
        ta_emb_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_x = ta_emb_x.unstack(emb_x)

        def _pretrain_recurrence(i, x_t, h_tm1, g_predictions):
            h_t = self.g_recurrent_unit(x_t, h_tm1)
            o_t = self.g_output_unit(h_t)
            g_predictions = g_predictions.write(i, o_t)
            x_tp1 = ta_emb_x.read(i)
            return i + 1, x_tp1, h_t, g_predictions

        _, _, _, self.g_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_pretrain_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.gather(self.g_embeddings, self.start_token),
                       self.h0, g_predictions))

        self.g_predictions = tf.reshape(
                self.g_predictions.stack(),
                [self.sequence_length, self.num_emb])

        # calculate discriminator loss
        self.d_gen_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.d_gen_predictions, labels=tf.zeros([self.sequence_length])))
        self.d_real_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.d_real_predictions, labels=tf.ones([self.sequence_length])))

        # calculate generator rewards and loss
        decays = tf.exp(tf.math.log(self.reward_gamma) * tf.compat.v1.to_float(tf.range(self.sequence_length)))
        rewards = _backwards_cumsum(decays * tf.sigmoid(self.d_gen_predictions),
                                    self.sequence_length)
        normalized_rewards = \
            rewards / _backwards_cumsum(decays, self.sequence_length) - self.expected_reward

        self.reward_loss = tf.reduce_mean(normalized_rewards ** 2)
        self.g_loss = \
            -tf.reduce_mean(tf.math.log(self.gen_o.stack()) * normalized_rewards)

        # pretraining loss
        self.pretrain_loss = \
            (-tf.reduce_sum(
                tf.one_hot(tf.compat.v1.to_int64(self.x),
                           self.num_emb, 1.0, 0.0) * tf.math.log(self.g_predictions))
             / self.sequence_length)

        # training updates
        d_opt = self.d_optimizer(self.learning_rate)
        g_opt = self.g_optimizer(self.learning_rate)
        pretrain_opt = self.g_optimizer(self.learning_rate)
        reward_opt = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate)

        self.d_gen_grad = tf.gradients(self.d_gen_loss, self.d_params)
        self.d_real_grad = tf.gradients(self.d_real_loss, self.d_params)
        self.d_gen_updates = d_opt.apply_gradients(zip(self.d_gen_grad, self.d_params))
        self.d_real_updates = d_opt.apply_gradients(zip(self.d_real_grad, self.d_params))

        self.reward_grad = tf.gradients(self.reward_loss, [self.expected_reward])
        self.reward_updates = reward_opt.apply_gradients(zip(self.reward_grad, [self.expected_reward]))

        self.g_grad = tf.gradients(self.g_loss, self.g_params)
        self.g_updates = g_opt.apply_gradients(zip(self.g_grad, self.g_params))

        self.pretrain_grad = tf.gradients(self.pretrain_loss, self.g_params)
        self.pretrain_updates = pretrain_opt.apply_gradients(zip(self.pretrain_grad, self.g_params))

    def generate(self, session):
        outputs = session.run(
                [self.gen_x],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs[0]

    def train_g_step(self, session):
        outputs = session.run(
                [self.g_updates, self.reward_updates, self.g_loss,
                 self.expected_reward, self.gen_x],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs

    def train_d_gen_step(self, session):
        outputs = session.run(
                [self.d_gen_updates, self.d_gen_loss],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs

    def train_d_real_step(self, session, x):
        outputs = session.run([self.d_real_updates, self.d_real_loss],
                              feed_dict={self.x: x})
        return outputs

    def pretrain_step(self, session, x):
        outputs = session.run([self.pretrain_updates, self.pretrain_loss, self.g_predictions],
                              feed_dict={self.x: x,
                                         self.h0: np.random.normal(size=self.hidden_dim)})
        return outputs

    def init_matrix(self, shape):
        return tf.compat.v1.random_normal(shape, stddev=0.1)

    def init_vector(self, shape):
        return tf.zeros(shape)

    def create_recurrent_unit(self, params):
        self.W_rec = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        params.append(self.W_rec)
        def unit(x_t, h_tm1):
            return h_tm1 + tf.reshape(tf.matmul(self.W_rec, tf.reshape(x_t, [self.emb_dim, 1])), [self.hidden_dim])
        return unit

    def create_output_unit(self, params, embeddings):
        self.W_out = tf.Variable(self.init_matrix([self.emb_dim, self.hidden_dim]))
        self.b_out1 = tf.Variable(self.init_vector([self.emb_dim, 1]))
        self.b_out2 = tf.Variable(self.init_vector([self.num_emb, 1]))
        params.extend([self.W_out, self.b_out1, self.b_out2])
        def unit(h_t):
            logits = tf.reshape(
                    self.b_out2 +
                    tf.matmul(embeddings,
                              tf.tanh(self.b_out1 +
                                      tf.matmul(self.W_out, tf.reshape(h_t, [self.hidden_dim, 1])))),
                    [1, self.num_emb])
            return tf.reshape(tf.nn.softmax(logits), [self.num_emb])
        return unit

    def create_classifier_unit(self, params):
        self.W_class = tf.Variable(self.init_matrix([1, self.hidden_dim]))
        self.b_class = tf.Variable(self.init_vector([1]))
        params.extend([self.W_class, self.b_class])
        def unit(h_t):
            return self.b_class + tf.matmul(self.W_class, tf.reshape(h_t, [self.hidden_dim, 1]))
        return unit

    def d_optimizer(self, *args, **kwargs):
        return tf.compat.v1.train.GradientDescentOptimizer(*args, **kwargs)

    def g_optimizer(self, *args, **kwargs):
        return tf.compat.v1.train.GradientDescentOptimizer(*args, **kwargs)


class GRU(RNN):

    def create_recurrent_unit(self, params):
        self.W_rx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.W_zx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.W_hx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.U_rh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        self.U_zh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        self.U_hh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        params.extend([
            self.W_rx, self.W_zx, self.W_hx,
            self.U_rh, self.U_zh, self.U_hh])

        def unit(x_t, h_tm1):
            x_t = tf.reshape(x_t, [self.emb_dim, 1])
            h_tm1 = tf.reshape(h_tm1, [self.hidden_dim, 1])
            r = tf.sigmoid(tf.matmul(self.W_rx, x_t) + tf.matmul(self.U_rh, h_tm1))
            z = tf.sigmoid(tf.matmul(self.W_zx, x_t) + tf.matmul(self.U_zh, h_tm1))
            h_tilda = tf.tanh(tf.matmul(self.W_hx, x_t) + tf.matmul(self.U_hh, r * h_tm1))
            h_t = (1 - z) * h_tm1 + z * h_tilda
            return tf.reshape(h_t, [self.hidden_dim])

        return unit

## Train

In [ ]:
def train_epoch(sess, trainable_model, num_iter,
                proportion_supervised, g_steps, d_steps,
                next_sequence, results, epoch, verify_sequence=None,
                words=None,
                proportion_generated=0.5):
    """Perform training for model.

    sess: tensorflow session
    trainable_model: the model
    num_iter: number of iterations
    proportion_supervised: what proportion of iterations should the generator
        be trained in a supervised manner (rather than trained via discriminator)
    g_steps: number of generator training steps per iteration
    d_steps: number of discriminator t raining steps per iteration
    next_sequence: function that returns a groundtruth sequence
    verify_sequence: function that checks a generated sequence, returning True/False
    words:  array of words (to map indices back to words)
    proportion_generated: what proportion of steps for the discriminator
        should be on artificially generated data

    """
    supervised_g_losses = [0]  # we put in 0 to avoid empty slices
    unsupervised_g_losses = [0]  # we put in 0 to avoid empty slices
    d_losses = [0]
    expected_rewards = [[0] * trainable_model.sequence_length]
    supervised_correct_generation = [0]
    unsupervised_correct_generation = [0]
    supervised_gen_x = None
    unsupervised_gen_x = None
    print('running %d iterations with %d g steps and %d d steps' % (num_iter, g_steps, d_steps))
    print('of the g steps, %.2f will be supervised' % proportion_supervised)
    for it in range(num_iter):
        for _ in range(g_steps):
            if random.random() < proportion_supervised:
                seq = next_sequence()
                _, g_loss, g_pred = trainable_model.pretrain_step(sess, seq)
                supervised_g_losses.append(g_loss)

                supervised_gen_x = np.argmax(g_pred, axis=1)
                if verify_sequence is not None:
                    supervised_correct_generation.append(
                        verify_sequence(supervised_gen_x))
            else:
                _, _, g_loss, expected_reward, unsupervised_gen_x = \
                    trainable_model.train_g_step(sess)
                expected_rewards.append(expected_reward)
                unsupervised_g_losses.append(g_loss)

                if verify_sequence is not None:
                    unsupervised_correct_generation.append(
                        verify_sequence(unsupervised_gen_x))

        for _ in range(d_steps):
            if random.random() < proportion_generated:
                seq = next_sequence()
                _, d_loss = trainable_model.train_d_real_step(sess, seq)
            else:
                _, d_loss = trainable_model.train_d_gen_step(sess)
            d_losses.append(d_loss)
    

    results['d_losses'][epoch] = np.mean(d_losses)
    results['supervised_g_losses'][epoch] = np.mean(supervised_g_losses)
    results['unsupervised_g_losses'][epoch] = np.mean(unsupervised_g_losses)
    results['supervised_generations'][epoch] = [words[x] if words else x for x in supervised_gen_x] if supervised_gen_x is not None else None
    results['unsupervised_generations'][epoch] = [words[x] if words else x for x in unsupervised_gen_x] if unsupervised_gen_x is not None else None
    results['rewards'][epoch] = np.mean(expected_rewards, axis=0)
    print('epoch statistics:')
    print('>>>> discriminator loss:', results['d_losses'][epoch])
    
    print('>>>> generator loss:', results['supervised_g_losses'][epoch], results['unsupervised_g_losses'][epoch])

    if verify_sequence is not None:
        print('>>>> correct generations (supervised, unsupervised):', np.mean(supervised_correct_generation), np.mean(unsupervised_correct_generation))
    print('>>>> sampled generations (supervised, unsupervised):',)
    print(results['supervised_generations'][epoch],)
    print(results['unsupervised_generations'][epoch])
    print('>>>> expected rewards:', results['rewards'][epoch])

## SeqGAN

In [ ]:
# HYPERPARAMETERS
EMB_DIM = 20
HIDDEN_DIM = 25
SEQ_LENGTH = 10
START_TOKEN = 0

EPOCH_ITER = 1000
CURRICULUM_RATE = 0.02  # how quickly to move from supervised training to unsupervised
TRAIN_ITER = 100000  # generator/discriminator alternating
D_STEPS = 3  # how many times to train the discriminator per generator step
SEED = 88




def tokenize(s):
    return [c for c in ' '.join(s.split())]


def get_data():
    if not os.path.isfile('sessions.pkl') or not os.path.isfile('all_tracks.pkl'):
        token_stream, all_tracks = get_sessions(100000)
        token_stream.to_pickle("sessions.pkl")
        output = open('all_tracks.pkl', 'wb')
        pickle.dump(all_tracks, output)
    else:
        print("Sessions exist, using sessions.pkl and all_tracks.pkl")
        token_stream = pd.read_pickle("sessions.pkl")
        all_tracks = pd.read_pickle("all_tracks.pkl")
    return token_stream, all_tracks


class BookGRU(GRU):

    def d_optimizer(self, *args, **kwargs):
        return tf.compat.v1.train.AdamOptimizer()  # ignore learning rate

    def g_optimizer(self, *args, **kwargs):
        return tf.compat.v1.train.AdamOptimizer()  # ignore learning rate


def get_trainable_model(num_emb):
    return BookGRU(
        num_emb, EMB_DIM, HIDDEN_DIM,
        SEQ_LENGTH, START_TOKEN)


def get_random_sequence(token_stream):
    """Returns random subsequence."""


    while True:
        row_idx = random.randint(0, len(token_stream)-1)
        if len(token_stream[row_idx]) >= SEQ_LENGTH:
            break

    start_idx = random.randint(0, len(token_stream[row_idx]) - SEQ_LENGTH)

    return token_stream[row_idx][start_idx:start_idx + SEQ_LENGTH]


def verify_sequence(three_grams, seq):
    """Not a true verification; only checks 3-grams."""
    for i in range(len(seq) - 3):
        if tuple(seq[i:i + 3]) not in three_grams:
            return False
    return True

def init_dict():
    results = {}
    results['d_losses'] = {}
    results['supervised_g_losses'] = {}
    results['unsupervised_g_losses'] = {}
    results['supervised_generations'] = {}
    results['unsupervised_generations'] = {}
    results['rewards'] = {}
    return results


def run():
    tf.compat.v1.disable_eager_execution()
    random.seed(SEED)
    np.random.seed(SEED)
    
    token_stream, all_tracks = get_data() # Read in data & create sessions
    results = init_dict() # initialise results dictionary
    
    # create words from all track_ids
    track_keys = []
    for key in all_tracks.keys():
        track_keys.append(key)
    words = track_keys

    # create index to word dicttionary for track_ids
    idx2word = {}
    for i in range(len(all_tracks)):
        idx2word[i] = all_tracks.get(i)['track_name']
    
    num_words = len(words)
    three_grams = {}
    count=0
    sec_count=0

    # create dictionary of 3-gram verification values
    for idx, row in token_stream.iteritems():
        sec_count += len(row)
        if len(row) > 3 :
            for i in range(len(row) - 3):
                three_grams[tuple(w for w in row[i:i + 3])] = True
        else : count += len(row)
    
    
    # print("Less than |3| = ", count)
    # print("Total count = ", sec_count)
    # print('num words', num_words)
    # print('stream length', len(token_stream))
    # print('distinct 3-grams', len(three_grams))

    trainable_model = get_trainable_model(num_words)
    sess = tf.compat.v1.Session()
    sess.run(tf.compat.v1.global_variables_initializer())
    start_time = time.time()
    print('Training...')
    for epoch in range(TRAIN_ITER // EPOCH_ITER):
        print('epoch', epoch)
        proportion_supervised = max(0.0, 1.0 - CURRICULUM_RATE * epoch)
        train_epoch(
            sess, trainable_model, EPOCH_ITER,
            proportion_supervised=proportion_supervised,
            g_steps=1, d_steps=D_STEPS,
            next_sequence=lambda: get_random_sequence(token_stream),
            verify_sequence=lambda seq: verify_sequence(three_grams, seq),
            words=words, results=results, epoch=epoch)

    print("Time taken: ", time.time()-start_time)

    # Save results for each value of D_STEPS
    with open('results_'+str(D_STEPS)+'.pkl', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    return token_stream, all_tracks

## Run SeqGAN

In [ ]:
token_stream, all_tracks = run()

## Visualisation

In [ ]:
def plotCurve(loss, legend, title):
    plt.clf()
    plt.plot(loss)
    plt.title('Learning Curve')
    plt.ylabel('NLL')
    plt.xlabel('Epoch')
    plt.legend(legend, loc='upper right')
    plt.savefig(title+".png", bbox_inches="tight", dpi=100)
    plt.show()
    
    
def plotBLEU(generations, title, legend):    
  plt.clf() 
  plt.plot(generations)
  plt.title(title)
  plt.ylabel('BLEU')
  plt.xlabel('Epoch')
  plt.legend(legend, loc='upper right')
  plt.savefig(title+".png", bbox_inches="tight", dpi=100)
  plt.show()
  

def plotSessions(results, label, filename):
  plt.clf()
  plt.ylabel("Number of Sessions")
  plt.xlabel("Session Length (minutes)")

  cmap = cm.get_cmap('viridis')
  bins = np.linspace(0, 50, 10)
  
  plt.style.use('seaborn-deep')
  plt.hist([results[5], results[35]], bins, label=label)
  plt.legend(loc='upper right')
  plt.savefig(filename, dpi=100)
  plt.show()

## Plot Session Lengths for varios values of Δt

In [ ]:
plotSessions([results[5], results[35]], 
             ['\u0394t = 5 minutes', '\u0394t = 35 minutes'], 
             "delta5_35_histogram.png")

plotSessions([results[10], results[20], results[30]], 
             ['\u0394t = 10 minutes', '\u0394t = 20 minutes', '\u0394t = 30 minutes'], 
             "delta10_20_30_histogram.png")

## Plot Learning Curve for various values of D_STEPS

In [ ]:

for D_STEPS in range(1,5):
    with open('results_'+str(D_STEPS)+'.pkl', 'rb') as handle:
        res = pickle.load(handle)
    x = []    
    for r in range(100):
        x.append(res['unsupervised_g_losses'][r])
    print("Loss for D_STEPS = {}: {}".format(D_STEPS, res['unsupervised_g_losses'][99]))
        
    plotCurve(x, ['D_STEPS = '+str(D_STEPS)], "CMON_"+str(D_STEPS))

## Plot BLEU Score for each generation for various values of D_STEPS

In [ ]:

for D_STEPS in range(1,5):
  with open('results_'+str(D_STEPS)+'.pkl', 'rb') as handle:
      results = pickle.load(handle)
  x = []
  for epoch in range(100):
    if results['unsupervised_generations'][epoch] is not None:
      x.append(sentence_bleu(token_stream, results['unsupervised_generations'][epoch], smoothing_function=SmoothingFunction().method1))
  
  plotBLEU(x, "BLEU_"+str(D_STEPS), "D_STEP = "+str(D_STEPS))


## Peer versus BLEU Score Graph

In [ ]:
x = []
with open('results_3.pkl', 'rb') as handle:
    results = pickle.load(handle)
  # print(epoch, type(results['unsupervised_generations'][epoch]))
    for epoch in range(96, 100):
        print("Epoch {}: {}".format(epoch, results['unsupervised_generations'][epoch]))
        for tr in results['unsupervised_generations'][epoch]:
            print("{} - {}".format(all_tracks[tr]['track_name'], all_tracks[tr]['artist']))
        print("{:2f}".format(sentence_bleu(token_stream, results['unsupervised_generations'][epoch], smoothing_function=SmoothingFunction().method4)))
        print()